In [ ]:
import os
from openai import OpenAI
import rich
from google.colab import userdata

In [ ]:
api_key = userdata.get('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"

openai = OpenAI(api_key=api_key)

**Sending Tools information into API calls**

# Chat Completion API

https://platform.openai.com/docs/guides/function-calling?api-mode=chat

Defining the structure schema of the function to be passed as a tool in the API.

In [ ]:
def get_weather_function_chat():
    return {
        "type": "function",
        "function": { # This property is removed from responses API
            "name": "get_weather",
            "description": "Get the weather for a location. Call this whenever you need to know the weather, for example when a customer asks 'What's the weather like in this city'",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and Countery e.g Karachi, Pakistan"
                    }
                },
                "required": ["location"],
                "additionalProperties": False
            },
            "strict": True
        }
    }

In [ ]:
response = openai.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "developer", "content": "You are a helpful assistant and provide update on weather in a city."},
        {"role": "user", "content": "What's the weather like in Karachi, Pakistan?"}
        # {"role": "user", "content": "NYC"}
    ],
    tools = [get_weather_function_chat()]
)

print(response.choices[0].message.content)
rich.print(response)
rich.print(response.choices[0].message.tool_calls)
print(response.choices[0].message.tool_calls[0].function.name)
print(response.choices[0].message.tool_calls[0].function.arguments)
print()
# Chat API provide value 'tool_calls' in finish_reason whenever a tool call is required
print("Finish Reason = ", response.choices[0].finish_reason)

None


ChatCompletion(
    id='chatcmpl-BF2MEyYDUcO3mAsLcr2kzzAn8AHEj',
    choices=[
        Choice(
            finish_reason='tool_calls',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                refusal=None,
                role='assistant',
                annotations=[],
                audio=None,
                function_call=None,
                tool_calls=[
                    ChatCompletionMessageToolCall(
                        id='call_auOwnnKfGdTvkPN1oyKAPjIH',
                        function=Function(arguments='{"location":"Karachi, Pakistan"}', name='get_weather'),
                        type='function'
                    )
                ]
            )
        )
    ],
    created=1742923682,
    model='gpt-4o-mini-2024-07-18',
    object='chat.completion',
    service_tier='default',
    system_fingerprint='fp_e4fa3702df',
    usage=CompletionUsage(
        completion_tokens=18,
        prompt_tokens=103,
        total_tokens=121,
        completion_tokens_details=CompletionTokensDetails(
            accepted_prediction_tokens=0,
            audio_tokens=0,
            reasoning_tokens=0,
            rejected_prediction_tokens=0
        ),
        prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
    )
)

[
    ChatCompletionMessageToolCall(
        id='call_auOwnnKfGdTvkPN1oyKAPjIH',
        function=Function(arguments='{"location":"Karachi, Pakistan"}', name='get_weather'),
        type='function'
    )
]

get_weather
{"location":"Karachi, Pakistan"}

Finish Reason =  tool_calls


# Responses API

https://platform.openai.com/docs/guides/function-calling?api-mode=responses

Schema strucute is different in Responses API

In [ ]:
def get_weather_function_response():
    return {
        "type": "function", # There is no function property in the response API
        "name": "get_weather",
        "description": "Get the weather for a location. Call this whenever you need to know the weather, for example when a customer asks 'What's the weather like in this city'",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and Countery e.g Karachi, Pakistan"
                }
            },
            "required": ["location"],
            "additionalProperties": False
        },
        "strict": True
    }

In [ ]:
response = openai.responses.create(
    model=MODEL,
    input=[{"role": "user", "content": "What is the weather like in Paris today?"}],
    tools = [get_weather_function_response()]
)

print(response.output_text)
rich.print(response)
rich.print("Status = ",response.status) # No indication of the tool call
rich.print(response.output[0]) # if you mention two cities in the input, you will get two outputs 0 and 1
print()
print("Name = ", response.output[0].name)
print("Type = ", response.output[0].type)
print("Arguments = ",response.output[0].arguments)

Response(
    id='resp_67e2e813e0748192811b20bcc60b080f06ab42833f632b5a',
    created_at=1742923795.0,
    error=None,
    incomplete_details=None,
    instructions=None,
    metadata={},
    model='gpt-4o-mini-2024-07-18',
    object='response',
    output=[
        ResponseFunctionToolCall(
            arguments='{"location":"Paris, France"}',
            call_id='call_B8R7yXrnbqAs6wpd0pCYhrZD',
            name='get_weather',
            type='function_call',
            id='fc_67e2e81502e481928b987e895f61fab706ab42833f632b5a',
            status='completed'
        )
    ],
    parallel_tool_calls=True,
    temperature=1.0,
    tool_choice='auto',
    tools=[
        FunctionTool(
            name='get_weather',
            parameters={
                'type': 'object',
                'properties': {
                    'location': {'type': 'string', 'description': 'City and Countery e.g Karachi, Pakistan'}
                },
                'required': ['location'],
                'additionalProperties': False
            },
            strict=True,
            type='function',
            description="Get the weather for a location. Call this whenever you need to know the weather, for 
example when a customer asks 'What's the weather like in this city'"
        )
    ],
    top_p=1.0,
    max_output_tokens=None,
    previous_response_id=None,
    reasoning=Reasoning(effort=None, generate_summary=None),
    status='completed',
    text=ResponseTextConfig(format=ResponseFormatText(type='text')),
    truncation='disabled',
    usage=ResponseUsage(
        input_tokens=299,
        input_tokens_details=InputTokensDetails(cached_tokens=0),
        output_tokens=17,
        output_tokens_details=OutputTokensDetails(reasoning_tokens=0),
        total_tokens=316
    ),
    user=None,
    store=True
)

Status =  completed

ResponseFunctionToolCall(
    arguments='{"location":"Paris, France"}',
    call_id='call_B8R7yXrnbqAs6wpd0pCYhrZD',
    name='get_weather',
    type='function_call',
    id='fc_67e2e81502e481928b987e895f61fab706ab42833f632b5a',
    status='completed'
)


Name =  get_weather
Type =  function_call
Arguments =  {"location":"Paris, France"}
